# Mesh generation for tutorial 05

In [ ]:
import os
import meshio
import pygmsh
from dolfinx import MPI
from dolfinx.io import XDMFFile
from dolfinx.plotting import plot

### Geometrical parameters

In [ ]:
r = 3
lcar = 1. / 4.

### Create mesh

In [ ]:
geom = pygmsh.built_in.Geometry()
c0 = geom.add_circle([0.0, 0.0, 0.0], r, lcar)
geom.add_physical(c0.plane_surface, label=0)
geom.add_physical(c0.line_loop.lines, label=1)
pygmsh_mesh = pygmsh.generate_mesh(geom)

### Save temporary mesh (pygmsh format)

In [ ]:
meshio.write("circle_tmp.xdmf", meshio.Mesh(
    points=pygmsh_mesh.points[:, :2],
    cells={"triangle": pygmsh_mesh.cells_dict["triangle"]}
))

### Save temporary boundaries (pygmsh format)

In [ ]:
meshio.write("circle_boundaries_tmp.xdmf", meshio.Mesh(
    points=pygmsh_mesh.points[:, :2],
    cells={"line": pygmsh_mesh.cells_dict["line"]},
    cell_data={"boundaries": [pygmsh_mesh.cell_data_dict["gmsh:physical"]["line"]]}
))

### Read back in mesh (dolfinx format)

In [ ]:
with XDMFFile(MPI.comm_world, "circle_tmp.xdmf", "r") as infile:
    mesh = infile.read_mesh(name="Grid")
    mesh.name = "mesh"

In [ ]:
plot(mesh)

### Read back in boundaries (dolfinx format)

In [ ]:
mesh.create_connectivity_all()
with XDMFFile(MPI.comm_world, "circle_boundaries_tmp.xdmf", "r") as infile:
    boundaries = infile.read_meshtags(mesh, name="Grid")
    boundaries.name = "boundaries"

### Save final mesh and boundaries (dolfinx format)

In [ ]:
with XDMFFile(MPI.comm_world, "circle.xdmf", "w") as file:
    file.write_mesh(mesh)
    file.write_meshtags(boundaries)

In [ ]:
os.remove("circle_tmp.xdmf")
os.remove("circle_tmp.h5")
os.remove("circle_boundaries_tmp.xdmf")
os.remove("circle_boundaries_tmp.h5")